In [ ]:
!pip install scikit-learn==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


In [ ]:
import sklearn
print(sklearn.__version__)

1.2.0


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, f1_score, confusion_matrix, roc_curve, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV



In [ ]:
df=pd.read_csv('get_around_pricing_project.csv')
df.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [ ]:
df.iloc[0,:]

Unnamed: 0                             0
model_key                        Citroën
mileage                           140411
engine_power                         100
fuel                              diesel
paint_color                        black
car_type                     convertible
private_parking_available           True
has_gps                             True
has_air_conditioning               False
automatic_car                      False
has_getaround_connect               True
has_speed_regulator                 True
winter_tires                        True
rental_price_per_day                 106
Name: 0, dtype: object

In [ ]:
df=df.drop('Unnamed: 0',axis=1)

In [ ]:
df.isnull().any()

model_key                    False
mileage                      False
engine_power                 False
fuel                         False
paint_color                  False
car_type                     False
private_parking_available    False
has_gps                      False
has_air_conditioning         False
automatic_car                False
has_getaround_connect        False
has_speed_regulator          False
winter_tires                 False
rental_price_per_day         False
dtype: bool

In [ ]:
# Separate target variable Y from features X
target_name ='rental_price_per_day'

print("Separating labels from features...")
Y = df.loc[:,target_name]
X = df.loc[:,[c for c in df.columns if c!=target_name]] 

Separating labels from features...


In [ ]:
# Divide dataset Train set & Test set 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=0)


In [ ]:
idx = 0
numeric_features = []
numeric_indices = []
categorical_features = []
categorical_indices = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
        numeric_indices.append(idx)
    else :
        categorical_features.append(i)
        categorical_indices.append(idx)

    idx = idx + 1

print('Found numeric features ', numeric_features,' at positions ', numeric_indices)
print('Found categorical features ', categorical_features,' at positions ', categorical_indices)

Found numeric features  ['mileage', 'engine_power']  at positions  [1, 2]
Found categorical features  ['model_key', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']  at positions  [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [ ]:
# Create pipeline for numeric features
numeric_features = X.select_dtypes([np.number]).columns
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
# Create pipeline for categorical features
categorical_features = X.select_dtypes("object").columns
categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first'))])

# Use ColumnTranformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
model_getaround= Pipeline(steps=[
("Preprocessing", preprocessor),
("Regressor",LinearRegression())
])

In [ ]:
model_getaround.fit(X_train, Y_train)

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['mileage', 'engine_power'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  Index(['model_key', 'fuel', 'paint_color', 'car_type'], dtype='object'))])),
                ('Regressor', LinearRegression())])

In [ ]:
predictions = model_getaround.predict(X_train)

In [ ]:
print(predictions)

[136.85028098 116.44359394 115.6176672  ... 117.43834795 103.84364586
 118.56353027]


In [ ]:
print("R2 score on training set : ", r2_score(Y_train, predictions))

R2 score on training set :  0.6764315098955815


In [ ]:
import joblib

In [ ]:
# dumping model
joblib.dump(model_getaround, "./model_1.joblib")

['./model_1.joblib']